# Algorithm

## 1. Load data

- Data structure: list of dictionary {"work_id", "work_description", "dependency", "component", "time_cost"}

## 2. Get all the relationship ("draw" the full map - refer here: https://miro.com/app/board/uXjVLEJLvHU=/?share_link_id=541552270638)

- Actually, it seems that this step is not necessary. Yesterday, I drawn the chart out using the relationships in the table in order to visualize the full nexus of work then to group and prioritize them

- When drawing the chart, I went down line by line and just drew without infering any new relationship. So it was just about direct relationship; when mapping out, all of these direct relationships between the nodes create a picture of a nexus and the indirect relationship and the structure of the network is revealed.

- No the above thinking is not correct. Actually, in the spreadsheet table, we know the full list of parents of each node but do not see the full list of children of each node --> so actually, drawing the map out and connecting the nodes create more information (more inference)!

- Procedure:
    + Must setup the table {"node_id", "parents", "children"} first
    + Go through each line of the loaded table
    + For each line, check the work_id first to update the parent list of it, then go through the parents and update the children list of it


## 3. Sort the work items ("coloring" - refer here: https://miro.com/app/board/uXjVLEJLvHU=/?share_link_id=541552270638)

- Iteration 1: 
   + Go through all the nodes. The nodes having no dependency/component will be assigned order = 1 (the order (color) data of these nodes will be saved to a dictionary name node_order)
   
- Iteration 2: 
   + Current order = 2
   + Go through all the nodes that have been colored (order-assigned), check the un-ordered (un-colored) children of these nodes.
      + If all the parent nodes of that child node have been order-assigned 1 --> update the order of the child node to 2
      + Else, that child node will not be colored in this round (because some of the work items that it depends on have not been completed so it cannot start this round)
      
- Iteration N (N>2):
   + Current order = N
   + Go through all the nodes that have been colored (order-assigned), check the un-ordered (un-colored) children of these nodes.
       + If all the parent nodes of that child node have been order-assigned an order which is less than N (can be N-1, N-2, anything but cannot be N) --> update the order of the child node to N
       + Else, that child node will not be colored in this round (because some of the work items that it depends on have not been completed so it cannot start this round)
       
- In order to perform this step:

    + Must know the children and parents of each node --> so still need to do step 2 --> a list of dictionary {"node_id", "parents", "children"}
    
    + At the start of each iteration, must know the color of each node/ the current order assigned to each node. During the iteration when going through the nodes, will update this color/order database --> data structure: a list of dictionary {"node_id", "order"}
    
    + Must write the read/write functions for these data strucure

In [193]:
import pandas as pd
import math
import json

In [194]:
# Helper functions

In [195]:
def string_to_list(input_string):
#     print(input_string)
    
    try:
        if input_string == '-' or input_string == '- ':
            return []
        else:
            return [item.strip() for item in input_string.split(',')]
    except:
        return []   

In [196]:
def convert_work_item_row(row):
    return {
        'work_id': row['work_id'],
        'work_description': row['work_description'],
        'dependency': string_to_list(row['dependency']),
        'component': string_to_list(row['component']),
        'time_cost': row['time_cost']
    }

In [197]:
class NodeRelationships:
    
    def __init__(self):
        self.table = []
        
    def is_node_in_the_table(self, node_id):
#         only count main node, children and parents of main nodes do not count
        for row in self.table:
            if row['id'] == node_id:
                return True
        return False
    
    def initiate_new_node(self, node_id):
        self.table.append(
            {"id": node_id,
             "parents": [],
             "children": []
            }
        )
        
    def display_node_info(self, node_id):
        for row in self.table:
            if row['id'] == node_id:
                print(row)
                break
                
    def display_full_table(self):
        print(self.table)
    
    def add_parent(self, node_id, parent_id):
        for row in self.table:
            if row['id'] == node_id:
                if not(parent_id in row['parents']): 
                       row['parents'].append(parent_id)
                       break
                else:
                       break
    
    def add_child(self, node_id, child_id):
        for row in self.table:
            if row['id'] == node_id:
                if not(child_id in row['children']): 
                       row['children'].append(child_id)
                       break
                else:
                       break
    
    def add_parent_list(self, node_id, parent_id_list):
        for parent_id in parent_id_list:
            self.add_parent(node_id, parent_id)
    
    def add_children_list(self, node_id, child_id_list):
        for child_id in child_id_list:
            self.add_child(node_id, child_id)
    
    def get_parents(self, node_id):
        if not self.is_node_in_the_table(node_id):
            raise Exception("Node {} not in the table!".format(node_id))
        else:
            for row in self.table:
                if row['id'] == node_id:
                    return row['parents']
    
    def get_children(self, node_id):
        if not self.is_node_in_the_table(node_id):
            raise Exception("Node {} not in the table!".format(node_id))
        else:
            for row in self.table:
                if row['id'] == node_id:
                    return row['children']

In [198]:
# test

# node_relationships = NodeRelationships()
# node_relationships.initiate_new_node("ID-005-497")
# node_relationships.add_parent("ID-005-497", "ID-018-195")
# node_relationships.add_child("ID-005-497", "ID-003-270")
# node_relationships.initiate_new_node("ID-006-400")
# node_relationships.add_parent_list("ID-006-400", ['ID-013-399', 'ID-014-944', 'ID-015-251', 'ID-016-980'])
# node_relationships.add_children_list("ID-006-400", ['yolo'])
# node_relationships.get_parents('ID-006-400')

In [282]:
class NodeOrders:
    
    def __init__(self):
        self.table = []

    def is_node_in_the_table(self, node_id):
#         only count main node, children and parents of main nodes do not count
        for row in self.table:
            if row['id'] == node_id:
                return True
        return False
    
    def initiate_new_node(self, node_id):
#         -1 for non-assigned order
        self.table.append(
            {"id": node_id,
             "order": -1
            }
        )
    
    def get_node_order(self, node_id):
        if not self.is_node_in_the_table(node_id):
            raise Exception("node_id {} not in the table yet!".format(node_id))
        else:
            for row in self.table:
                if row['id'] == node_id:
                    return row['order']

    def display_node_info(self, node_id):
        for row in self.table:
            if row['id'] == node_id:
                print(row)
                break

    def display_full_table(self):
        print(self.table)
    
    def assign_order(self, node_id, order):
        if not self.is_node_in_the_table(node_id):
            self.initiate_new_node(node_id)
        for row in self.table:
            if row['id'] == node_id:
                row['order'] = order
                break

    def get_ordered_nodes(self):
        result = []
        for row in self.table:
            if row["order"] > 0:
                result.append(row["id"])
        return result

    
    def get_unordered_nodes(self):
        result = []
        for row in self.table:
            if row["order"] == -1:
                result.append(row["id"])
        return result
    
    def get_nodes_of_order(self, order):
        result = []
        for row in self.table:
            if row["order"] == order:
                result.append(row["id"])
        return result
    
    def get_highest_order(self):
        result = -1
        for row in self.table:
            if row['order'] > result:
                result = row['order']
        return result

In [283]:
# test

# node_orders = NodeOrders()
# node_orders.assign_order("ID-003-270", 1)
# node_orders.assign_order("ID-003-270", 2)
# node_orders.initiate_new_node("ID-020-196")
# node_orders.assign_order("ID-020-196", 1)
# node_orders.initiate_new_node("ID-058-779")
# node_orders.display_full_table()
# node_orders.get_unordered_nodes()
# node_orders.get_node_order("ID-003-270")
# node_orders.get_unordered_nodes()

In [284]:
# Step 1

In [285]:
df_work_item_list = pd.read_csv("input/work_item_list.csv")

In [286]:
df_work_item_list

,work_id,work_description,dependency,component,time_cost
0,ID-002-528,slide 0 (cover slide - cover): craft the whole...,-,-,5.0
1,ID-003-270,slide 1 (table of content - table of content -...,-,-,8.0
2,ID-004-742,slide 2 (executive summary - section intro): c...,-,-,5.0
3,ID-005-497,slide 3 (executive summary - main content - 20...,"ID-018-195, ID-019-410",-,30.0
4,ID-006-400,slide 3 (executive summary - main content - 20...,"ID-013-399, ID-014-944, ID-015-251, ID-016-980",-,30.0
...,...,...,...,...,...
96,ID-098-535,slide 16 (2025 targeted genres - main content ...,ID-097-378,-,15.0
97,ID-099-829,slide 16 (2025 targeted genres - main content ...,ID-069-498,-,0.0
98,ID-100-476,slide 16 (2025 targeted genres - main content ...,ID-076-997,-,30.0
99,ID-101-626,slide 16 (2025 targeted genres - main content ...,ID-080-677,-,45.0


In [287]:
table_work_item_info = df_work_item_list.to_dict(orient='records')

In [288]:
table_work_item_info

[{'work_id': 'ID-002-528',
  'work_description': 'slide 0 (cover slide - cover): craft the whole slide',
  'dependency': '-',
  'component': '-',
  'time_cost': 5.0},
 {'work_id': 'ID-003-270',
  'work_description': 'slide 1 (table of content - table of content - table of content): craft the whole slide',
  'dependency': '-',
  'component': '-',
  'time_cost': 8.0},
 {'work_id': 'ID-004-742',
  'work_description': 'slide 2 (executive summary - section intro): craft the whole slide',
  'dependency': '-',
  'component': '-',
  'time_cost': 5.0},
 {'work_id': 'ID-005-497',
  'work_description': "slide 3 (executive summary - main content - 2024 review): add remark on 2024's targeted genres' performance ",
  'dependency': 'ID-018-195, ID-019-410',
  'component': '-',
  'time_cost': 30.0},
 {'work_id': 'ID-006-400',
  'work_description': 'slide 3 (executive summary - main content - 2024 review): add remark on top-performing genres and new games of 2024',
  'dependency': 'ID-013-399, ID-014-9

In [289]:
table_work_item_info = list(map(lambda x: convert_work_item_row(x), table_work_item_info))

In [290]:
table_work_item_info

[{'work_id': 'ID-002-528',
  'work_description': 'slide 0 (cover slide - cover): craft the whole slide',
  'dependency': [],
  'component': [],
  'time_cost': 5.0},
 {'work_id': 'ID-003-270',
  'work_description': 'slide 1 (table of content - table of content - table of content): craft the whole slide',
  'dependency': [],
  'component': [],
  'time_cost': 8.0},
 {'work_id': 'ID-004-742',
  'work_description': 'slide 2 (executive summary - section intro): craft the whole slide',
  'dependency': [],
  'component': [],
  'time_cost': 5.0},
 {'work_id': 'ID-005-497',
  'work_description': "slide 3 (executive summary - main content - 2024 review): add remark on 2024's targeted genres' performance ",
  'dependency': ['ID-018-195', 'ID-019-410'],
  'component': [],
  'time_cost': 30.0},
 {'work_id': 'ID-006-400',
  'work_description': 'slide 3 (executive summary - main content - 2024 review): add remark on top-performing genres and new games of 2024',
  'dependency': ['ID-013-399', 'ID-014-9

In [291]:
# Step 2

In [292]:
node_relationships = NodeRelationships()

In [293]:
for row in table_work_item_info:
    
    main_node = row['work_id']
    
    main_node_parents = row['dependency'] + row['component'] 
    
    # process main node
    
    if not node_relationships.is_node_in_the_table(main_node):
        node_relationships.initiate_new_node(main_node)
        print("main node: {}".format(main_node))
    
    # add parents to main node
    node_relationships.add_parent_list(main_node, main_node_parents)
    
    # process main node's parents (add main node as child to them)
    for node in main_node_parents:
        if not node_relationships.is_node_in_the_table(node):
            node_relationships.initiate_new_node(node)
            print("parent node: {}".format(node))
        node_relationships.add_child(node, main_node)

main node: ID-002-528
main node: ID-003-270
main node: ID-004-742
main node: ID-005-497
parent node: ID-018-195
parent node: ID-019-410
main node: ID-006-400
parent node: ID-013-399
parent node: ID-014-944
parent node: ID-015-251
parent node: ID-016-980
main node: ID-007-148
main node: ID-008-662
main node: ID-009-878
parent node: ID-037-380
main node: ID-010-747
main node: ID-011-599
main node: ID-012-524
parent node: ID-047-939
parent node: ID-048-126
parent node: ID-062-375
parent node: ID-063-325
parent node: ID-069-498
parent node: ID-070-555
parent node: ID-071-274
parent node: ID-072-840
main node: ID-017-925
main node: ID-020-312
main node: ID-021-108
main node: ID-022-200
main node: ID-023-455
main node: ID-024-388
main node: ID-025-803
main node: ID-026-947
main node: ID-027-918
main node: ID-028-310
main node: ID-029-120
parent node: ID-083-154
parent node: ID-084-311
parent node: ID-085-702
main node: ID-030-975
main node: ID-031-676
main node: ID-032-983
main node: ID-033-

In [294]:
print(json.dumps(node_relationships.table, indent=4))

[
    {
        "id": "ID-002-528",
        "parents": [],
        "children": []
    },
    {
        "id": "ID-003-270",
        "parents": [],
        "children": []
    },
    {
        "id": "ID-004-742",
        "parents": [],
        "children": []
    },
    {
        "id": "ID-005-497",
        "parents": [
            "ID-018-195",
            "ID-019-410"
        ],
        "children": [
            "ID-007-148",
            "ID-008-662"
        ]
    },
    {
        "id": "ID-018-195",
        "parents": [
            "ID-017-925"
        ],
        "children": [
            "ID-005-497",
            "ID-020-312"
        ]
    },
    {
        "id": "ID-019-410",
        "parents": [
            "ID-017-925"
        ],
        "children": [
            "ID-005-497",
            "ID-020-312"
        ]
    },
    {
        "id": "ID-006-400",
        "parents": [
            "ID-013-399",
            "ID-014-944",
            "ID-015-251",
            "ID-016-980"
        ],

In [295]:
# Step 3

In [296]:
node_orders = NodeOrders()

In [297]:
# Initiate the color table with all nodes uncolored

for row in node_relationships.table:
    node_orders.initiate_new_node(row['id'])

In [298]:
# Iteration 1

# Go through all the nodes. The nodes having no dependency/component will be assigned order = 1 
# (the order (color) data of these nodes will be saved to a dictionary name node_order)

current_order = 1

for row in node_relationships.table:
    if row["parents"] == []:
        node_orders.assign_order(row['id'], current_order)

In [299]:
node_orders.get_ordered_nodes()

['ID-002-528',
 'ID-003-270',
 'ID-004-742',
 'ID-012-524',
 'ID-025-803',
 'ID-026-947',
 'ID-028-310',
 'ID-038-716',
 'ID-041-402',
 'ID-044-498',
 'ID-054-887',
 'ID-058-363',
 'ID-064-653',
 'ID-075-266',
 'ID-079-466',
 'ID-093-937']

In [300]:
# Iteration N (N>=2):

# Current order = N
# Go through all the nodes that have been colored (order-assigned), 
# check the un-ordered (un-colored) children of these nodes.
# If all the parent nodes of that child node have been order-assigned an order 
# which is less than N (can be N-1, N-2, anything but cannot be N) --> update the order of the child node to N
# Else, that child node will not be colored in this round 
# (because some of the work items that it depends on have not been completed so it cannot start this round)

current_order = 1

while node_orders.get_unordered_nodes() != []:
    
    current_order += 1
    
    colored_nodes = node_orders.get_ordered_nodes()
    
    for node in colored_nodes:
        
        current_node_children = node_relationships.get_children(node)
        
        for child_node in current_node_children:
            
            child_node_order = node_orders.get_node_order(child_node)
            
            if child_node_order == -1:
                
                parent_nodes_of_the_child = node_relationships.get_parents(child_node)
                
                to_color = True
                
                for parent_node in parent_nodes_of_the_child:
                    
                    parent_node_order = node_orders.get_node_order(parent_node)
                    
                    if parent_node_order == -1 or parent_node_order == current_order:
                        
                        to_color = False
                        
                if to_color:
                    
                    node_orders.assign_order(child_node, current_order)

In [301]:
node_orders.table

[{'id': 'ID-002-528', 'order': 1},
 {'id': 'ID-003-270', 'order': 1},
 {'id': 'ID-004-742', 'order': 1},
 {'id': 'ID-005-497', 'order': 20},
 {'id': 'ID-018-195', 'order': 19},
 {'id': 'ID-019-410', 'order': 19},
 {'id': 'ID-006-400', 'order': 18},
 {'id': 'ID-013-399', 'order': 15},
 {'id': 'ID-014-944', 'order': 17},
 {'id': 'ID-015-251', 'order': 10},
 {'id': 'ID-016-980', 'order': 11},
 {'id': 'ID-007-148', 'order': 21},
 {'id': 'ID-008-662', 'order': 22},
 {'id': 'ID-009-878', 'order': 21},
 {'id': 'ID-037-380', 'order': 20},
 {'id': 'ID-010-747', 'order': 21},
 {'id': 'ID-011-599', 'order': 22},
 {'id': 'ID-012-524', 'order': 1},
 {'id': 'ID-047-939', 'order': 13},
 {'id': 'ID-048-126', 'order': 14},
 {'id': 'ID-062-375', 'order': 15},
 {'id': 'ID-063-325', 'order': 16},
 {'id': 'ID-069-498', 'order': 6},
 {'id': 'ID-070-555', 'order': 7},
 {'id': 'ID-071-274', 'order': 8},
 {'id': 'ID-072-840', 'order': 9},
 {'id': 'ID-017-925', 'order': 18},
 {'id': 'ID-020-312', 'order': 20},


In [302]:
node_orders.get_nodes_of_order(7)

['ID-070-555', 'ID-057-425', 'ID-099-829']

In [303]:
node_orders.get_highest_order()

23

In [304]:
for i in range(1,24):
    print("Order {}:".format(i))
    print(node_orders.get_nodes_of_order(i))
    print()

Order 1:
['ID-002-528', 'ID-003-270', 'ID-004-742', 'ID-012-524', 'ID-025-803', 'ID-026-947', 'ID-028-310', 'ID-038-716', 'ID-041-402', 'ID-044-498', 'ID-054-887', 'ID-058-363', 'ID-064-653', 'ID-075-266', 'ID-079-466', 'ID-093-937']

Order 2:
['ID-027-918', 'ID-059-251', 'ID-073-450', 'ID-076-997', 'ID-080-677', 'ID-091-105', 'ID-094-369']

Order 3:
['ID-060-849', 'ID-074-451', 'ID-081-480', 'ID-095-734', 'ID-100-476', 'ID-101-626']

Order 4:
['ID-061-296', 'ID-077-281', 'ID-082-718', 'ID-088-916']

Order 5:
['ID-055-848', 'ID-078-914', 'ID-089-723']

Order 6:
['ID-069-498', 'ID-084-311', 'ID-056-519']

Order 7:
['ID-070-555', 'ID-057-425', 'ID-099-829']

Order 8:
['ID-071-274', 'ID-052-345', 'ID-102-525']

Order 9:
['ID-072-840', 'ID-096-214', 'ID-053-731']

Order 10:
['ID-015-251', 'ID-097-378', 'ID-049-855']

Order 11:
['ID-016-980', 'ID-021-108', 'ID-098-535', 'ID-050-720', 'ID-065-906', 'ID-090-485']

Order 12:
['ID-022-200', 'ID-033-245', 'ID-051-563', 'ID-066-826', 'ID-086-474'

In [305]:
df_work_and_order = pd.DataFrame(node_orders.table)

In [306]:
df_work_and_order

,id,order
0,ID-002-528,1
1,ID-003-270,1
2,ID-004-742,1
3,ID-005-497,20
4,ID-018-195,19
...,...,...
96,ID-092-720,12
97,ID-099-829,7
98,ID-100-476,3
99,ID-101-626,3


In [307]:
df_work_and_order.to_csv("output/work_order.csv")